In [1]:
!curl -O https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3832  100  3832    0     0  11169      0 --:--:-- --:--:-- --:--:-- 11172


In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [34]:
import os
#os.environ["OPENAI_API_KEY"] = ""

from openai import OpenAI

In [35]:
client = OpenAI()

In [36]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"Yes, in many cases, you can still enroll in a course even after it has started. However, the specifics can depend on a variety of factors such as the institution offering the course, the type of course, and how far into the term it is. Here are some steps you can take:\n\n1. **Check the Enrollment Deadline**: Sometimes institutions have a cut-off date for adding courses. Check the course catalog or academic calendar for this information.\n\n2. **Contact the Instructor**: Reaching out directly to the instructor can be very effective. Explain your situation and express your interest in catching up on any missed work.\n\n3. **Contact Academic Advising**: Advisors can offer guidance on whether late enrollment is possible and what steps you need to take.\n\n4. **Complete Any Required Paperwork**: Some institutions may require you to fill out specific forms for late enrollment.\n\n5. **Assess the Coursework**: Make sure to review the syllabus and any materials provided so far to determine i

In [37]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [61]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    print(len(prompt))
    return prompt

In [62]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [63]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [64]:
rag(query)

1957


'To run Kafka, you can follow these steps based on the provided CONTEXT for different scenarios:\n\n1. **Java Kafka Producer/Consumer/KStreams**:\n   Navigate to your project directory and execute the following command:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **Python Kafka with Virtual Environment**:\n   First, create and activate a virtual environment, and install the necessary dependencies:\n   ```sh\n   python -m venv env\n   source env/bin/activate  # For MacOS/Linux\n   # or\n   env\\Scripts\\activate  # For Windows\n\n   pip install -r ../requirements.txt\n   ```\n\n   After activating the virtual environment, run your Python Kafka producer/consumer code within this environment.\n\nIf you need additional context on solving specific errors or installing other dependencies, refer to the provided FAQ entries.'

In [65]:
rag('the course has already started, can I still enroll?')

2189


"Yes, you can still enroll in the course even after it has started. However, you should be aware that there will be deadlines for submitting the final projects, so it's important not to leave everything until the last minute."

In [66]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [67]:
from elasticsearch import Elasticsearch

In [68]:
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [71]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [72]:
from tqdm.auto import tqdm

In [73]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 322.13it/s]


In [74]:
query = 'How do I execute a command in a running docker container?'

In [75]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    print(response)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [76]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    print(len(encoding.encode(prompt)))
    answer = llm(prompt)
    return answer

In [77]:
rag(query)

{'took': 689, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1035, 'relation': 'eq'}, 'max_score': 84.220634, 'hits': [{'_index': 'course-questions', '_id': 'rtpjQ5ABPWEJPAPRkbFO', '_score': 84.220634, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'YtpkQ5ABPWEJPAPR9bVo', '_score': 84.220634, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entryp

'To execute a command in a running Docker container, you can follow these steps:\n\n1. Identify the container ID of the running container:\n   ```bash\n   docker ps\n   ```\n   \n2. Execute a command in the specific container using the container ID:\n   ```bash\n   docker exec -it <container-id> bash\n   ```\n\nThis will open an interactive bash session inside your running container, allowing you to run commands as needed.'

In [70]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")